# Scanners

In [1]:
from ib_insync import *
util.startLoop()  

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=9)

<IB connected to 127.0.0.1:7497 clientId=9>

## Basic Scanner

To create a scanner create a `ScannerSubscription` option to submit to the `reqScannerData` method. For any scanner to work it must at least have the `instrument`, `locationCode`, and `scanCode` parameters filled.

Additionally, the `ScannerSubscription` has other filters directly submitted like `abovePrice` and `aboveVolume`. Check [here](https://github.com/erdewit/ib_insync/blob/master/ib_insync/objects.py) for a full list of the default filters.

In [2]:
sub = ScannerSubscription(instrument='STK', 
                    locationCode='STK.US.MAJOR', 
                    scanCode='TOP_PERC_GAIN', 
                    abovePrice=500, 
                    aboveVolume=1000000, 
                   )

scanData = ib.reqScannerData(sub, [])

ib.sleep(0.1)
ib.cancelScannerSubscription(scanData)

Error 162, reqId 3: Historical Market Data Service error message:API scanner subscription cancelled: 3
Error 365, reqId 3: No scanner subscription found for ticker id:3


The scanner will return a list of contracts but no other data. In this case we can only be sure that these stocks had a price over 500 and volume over 1,000,000 when the scan was performed.

We can loop through these contracts to find more information. For example, lets get the number of public floating shares for each contract in our scanner. 

In [3]:
from bs4 import BeautifulSoup

def getFloat(contract):

    fundamentals = ib.reqFundamentalData(contract, 'ReportSnapshot')
    fundamentals = BeautifulSoup(fundamentals, 'xml')

    qfloat = float(fundamentals.find_all('SharesOut')[0]['TotalFloat'])
    
    return qfloat
    
for scan in scanData: 
    try: 
        contract = scan.contractDetails.contract
        print('The public float for',contract.symbol,'is',getFloat(contract))
    except: 
        print('Public float not available for',contract.symbol)

The public float for TSLA is 142465943.0
The public float for GOOGL is 623218570.0


Error 430, reqId 6: We are sorry, but fundamentals data for the security specified is not available.failed to fetch, contract: Contract(secType='STK', conId=208813720, symbol='GOOG', exchange='SMART', currency='USD', localSymbol='GOOG', tradingClass='NMS')


Public float not available for GOOG
The public float for AMZN is 420206723.0


As you can see, not all contracts returned by the scanner will be valid for your purposes. Make sure you include checks when processing the scanner results. 

## Scanner Parameters

The scanner parameters map directly to the options available through the TWS "Advanced Market Scanner." You can check all the scanner parameters values available

In [4]:
allParams = ib.reqScannerParameters()
print(allParams[:3000]) # only print 3,000 characters to keep it short 

<?xml version="1.0" encoding="UTF-8"?>
<ScanParameterResponse>
	<InstrumentList varName="instrumentList">
		<Instrument>
			<name>US Stocks</name>
			<type>STK</type>
			<filters>AFTERHRSCHANGEPERC,AVGOPTVOLUME,AVGPRICETARGET,AVGRATING,AVGTARGET2PRICERATIO,AVGVOLUME,AVGVOLUME_USD,CHANGEOPENPERC,CHANGEPERC,EMA_20,EMA_50,EMA_100,EMA_200,PRICE_VS_EMA_20,PRICE_VS_EMA_50,PRICE_VS_EMA_100,PRICE_VS_EMA_200,DAYSTOCOVER,DIVIB,DIVYIELD,DIVYIELDIB,FEERATE,FIRSTTRADEDATE,GROWTHRATE,HALTED,HASOPTIONS,HISTDIVIB,HISTDIVYIELDIB,IMBALANCE,IMBALANCEADVRATIOPERC,IMPVOLAT,IMPVOLATOVERHIST,INSIDEROFFLOATPERC,INSTITUTIONALOFFLOATPERC,MACD,MACD_SIGNAL,MACD_HISTOGRAM,MKTCAP,MKTCAP_USD,NEXTDIVAMOUNT,NEXTDIVDATE,NUMPRICETARGETS,NUMRATINGS,NUMSHARESINSIDER,NUMSHARESINSTITUTIONAL,NUMSHARESSHORT,OPENGAPPERC,OPTVOLUME,OPTVOLUMEPCRATIO,PERATIO,PPO,PPO_SIGNAL,PPO_HISTOGRAM,PRICE,PRICE2BK,PRICE2TANBK,PRICERANGE,PRICE_USD,QUICKRATIO,REBATERATE,REGIMBALANCE,REGIMBALANCEADVRATIOPERC,RETEQUITY,SHORTABLESHARES,SHORTOFFLOAT

<br>For every parameter you can see the <name> which is how it appears in TWS and the <type> which is how you would submit it to the `ScannerSubscription` object. 

In the above example you can see that each instrument also shows you which filters are applicable to it. If we want to use any of these filter options we need to use the `TagValue` class and the `scannerSubscriptionFilterOptions` option. In this case lets use the "CHANGEPERC" filter to filter out stocks that have not moved more than 2% since yesterdays close. 

In [5]:
sub = ScannerSubscription(instrument='STK',           # Submit a scanner subscription for stocks 
                    locationCode='STK.US.MAJOR',      # Use only US-Major stock exchanges
                    scanCode='TOP_PERC_GAIN',         # Use the Top % Gainers scanner code 
                    abovePrice=5,                     # Any stock above $5 
                    aboveVolume=100000,               # and above 100,000 volume traded so far today 
                   )

# Create a list of tag values with the option and optionvalue 
tagValues = [TagValue("changePercAbove", "20")]

# Submit the tags when requesting the scanner data, don't forget to include the [] in the second argument
scanData = ib.reqScannerData(sub, [], scannerSubscriptionFilterOptions=tagValues)

ib.sleep(0.1)
ib.cancelScannerSubscription(scanData)

# Print out the sorted sumbols for the returned contracts 
symbols = [scan.contractDetails.contract.symbol for scan in scanData]
sorted(symbols)

Error 162, reqId 8: Historical Market Data Service error message:API scanner subscription cancelled: 8


['CRTX', 'GLUU', 'UIS']

Error 365, reqId 8: No scanner subscription found for ticker id:8


Note that in order to submit the "CHANGEPERC" filter I had to change it. 
- First I had to put it in [camelCase](https://en.wikipedia.org/wiki/Camel_case) format.
- Second, I had to add an "Above" suffix to tell the filter that I wanted everythin above that value. I could have changed this to "Below"

Anything scanner you can create in TWS can be submitted through the API. The `scanCode` parameter maps directly to the "Parameter" window in the TWS "Advanced Market Scanner." We can verify this by printing out the `scanCode` values available. 

In [6]:
paramSoup = BeautifulSoup(allParams, 'xml')
[sc.text for sc in paramSoup.find_all('scanCode')[:20]] # just print out 20 items for brevety

['SCAN_etfAssets_DESC',
 'SCAN_etfAssets_ASC',
 'SCAN_etfExpense_DESC',
 'SCAN_etfExpense_ASC',
 'SCAN_etfALTAR_DESC',
 'SCAN_etfALTAR_ASC',
 'SCAN_etfAvgALTAR_DESC',
 'SCAN_etfAvgALTAR_ASC',
 'SCAN_etfTRytd_DESC',
 'SCAN_etfTRytd_ASC',
 'SCAN_etfTR1yr_DESC',
 'SCAN_etfTR1yr_ASC',
 'SCAN_etfTR5yr_DESC',
 'SCAN_etfTR5yr_ASC',
 'SCAN_etfTR10yr_DESC',
 'SCAN_etfTR10yr_ASC',
 'SCAN_etfTRIncep_DESC',
 'SCAN_etfTRIncep_ASC',
 'SCAN_etfTrackingDiffPct_DESC',
 'SCAN_etfTrackingDiffPct_ASC']

## Other Products

You are not limited to stocks.

In [7]:
[i.find_all('name')[0].text for i in paramSoup.find_all('Instrument')][:10]

['US Stocks',
 'US Equity ETFs',
 'US Fixed Income ETFs',
 'US Futures',
 'US Indexes',
 'US SSFs',
 'Corporate Bonds',
 'US CDs',
 'US Agency Bonds',
 'US Treasuries']

In [8]:
sub = ScannerSubscription(instrument='FUT.US',
                          locationCode='FUT.GLOBEX', 
                          scanCode='MOST_ACTIVE')

scanData = ib.reqScannerData(sub, [])
contracts = [scan.contractDetails.contract for scan in scanData]
contracts[:5]

[Contract(secType='FUT', conId=346577697, symbol='ES', lastTradeDateOrContractMonth='20200320', exchange='GLOBEX', currency='USD', localSymbol='ESH0', tradingClass='ES'),
 Contract(secType='FUT', conId=346577750, symbol='NQ', lastTradeDateOrContractMonth='20200320', exchange='GLOBEX', currency='USD', localSymbol='NQH0', tradingClass='NQ'),
 Contract(secType='FUT', conId=362698381, symbol='MES', lastTradeDateOrContractMonth='20200320', exchange='GLOBEX', currency='USD', localSymbol='MESH0', tradingClass='MES'),
 Contract(secType='FUT', conId=362702255, symbol='MNQ', lastTradeDateOrContractMonth='20200320', exchange='GLOBEX', currency='USD', localSymbol='MNQH0', tradingClass='MNQ'),
 Contract(secType='FUT', conId=82252724, symbol='GE', lastTradeDateOrContractMonth='20201214', exchange='GLOBEX', currency='USD', localSymbol='GEZ0', tradingClass='GE')]

Error 162, reqId 9: Historical Market Data Service error message:API scanner subscription cancelled: 9


In [9]:
ib.disconnect()